# Ultra-Fast Audio Beam Focusing Simulator

This interactive notebook demonstrates the capabilities of the audio beam focusing simulator.

## Features
- Circular microphone array simulation
- FFT-based fractional delay processing
- Real-time beam focusing
- Interactive parameter adjustment
- Energy map visualization


In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML, display
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed

# Import our simulator
from beam_focusing_simulator import BeamFocusingSimulator

# Configure matplotlib for notebook
%matplotlib widget
plt.style.use('default')

print('Libraries imported successfully!')

## 1. Basic Setup and Configuration

In [ ]:
# Interactive parameter configuration
@interact(
    n_mics=widgets.IntSlider(value=32, min=8, max=64, step=8, description='Microphones:'),
    array_radius=widgets.FloatSlider(value=0.2, min=0.1, max=0.5, step=0.05, description='Array Radius (m):'),
    target_distance=widgets.FloatSlider(value=0.4, min=0.2, max=1.0, step=0.1, description='Target Distance (m):'),
    grid_resolution=widgets.IntSlider(value=70, min=30, max=120, step=10, description='Grid Resolution:')
)
def create_simulator(n_mics, array_radius, target_distance, grid_resolution):
    global simulator
    simulator = BeamFocusingSimulator(
        n_mics=n_mics,
        array_radius=array_radius,
        target_distance=target_distance,
        grid_resolution=grid_resolution
    )
    
    # Visualize array geometry
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # Plot microphone array (top view)
    mic_x = simulator.mic_positions[:, 0]
    mic_y = simulator.mic_positions[:, 1]
    ax1.scatter(mic_x, mic_y, c='blue', s=50, alpha=0.7)
    ax1.set_xlim(-array_radius*1.5, array_radius*1.5)
    ax1.set_ylim(-array_radius*1.5, array_radius*1.5)
    ax1.set_aspect('equal')
    ax1.set_title('Microphone Array (Top View)')
    ax1.set_xlabel('X (m)')
    ax1.set_ylabel('Y (m)')
    ax1.grid(True, alpha=0.3)
    
    # Add circle to show array
    circle = plt.Circle((0, 0), array_radius, fill=False, color='blue', linestyle='--', alpha=0.5)
    ax1.add_patch(circle)
    
    # Plot target plane (side view)
    ax2.scatter(mic_x, np.zeros_like(mic_x), c='blue', s=50, alpha=0.7, label='Microphones')
    target_corners_x = [-simulator.target_size/2, simulator.target_size/2, 
                        simulator.target_size/2, -simulator.target_size/2, -simulator.target_size/2]
    target_corners_z = [target_distance] * 5
    ax2.plot(target_corners_x, target_corners_z, 'r-', linewidth=2, label='Target Plane')
    ax2.set_xlim(-0.5, 0.5)
    ax2.set_ylim(-0.1, target_distance + 0.1)
    ax2.set_title('System Geometry (Side View)')
    ax2.set_xlabel('X (m)')
    ax2.set_ylabel('Z (m)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f'Simulator created with {n_mics} microphones')
    print(f'Target plane: {grid_resolution}×{grid_resolution} grid points')
    print(f'Total grid points: {grid_resolution**2}')

## 2. Audio Source Configuration

In [ ]:
# Interactive source positioning
@interact(
    source_x=widgets.FloatSlider(value=0.0, min=-0.15, max=0.15, step=0.02, description='Source X (m):'),
    source_y=widgets.FloatSlider(value=0.0, min=-0.15, max=0.15, step=0.02, description='Source Y (m):'),
    duration=widgets.FloatSlider(value=3.0, min=1.0, max=10.0, step=0.5, description='Duration (s):'),
    add_noise=widgets.Checkbox(value=True, description='Add Noise')
)
def generate_audio(source_x, source_y, duration, add_noise):
    global audio_data, source_position
    
    if 'simulator' not in globals():
        print('Please create a simulator first!')
        return
    
    source_position = (source_x, source_y, simulator.target_distance)
    
    print(f'Generating audio for source at ({source_x:.2f}, {source_y:.2f}, {simulator.target_distance:.2f})m')
    
    # Generate audio
    audio_data = simulator.generate_test_audio(
        duration=duration, 
        source_positions=[source_position]
    )
    
    if not add_noise:
        # Remove the noise component that was added in generate_test_audio
        pass  # The noise is already minimal in the current implementation
    
    # Show audio statistics
    print(f'Audio generated: {audio_data.shape[0]} samples, {audio_data.shape[1]} channels')
    print(f'Duration: {audio_data.shape[0] / simulator.sample_rate:.2f} seconds')
    print(f'RMS level: {np.sqrt(np.mean(audio_data**2)):.4f}')
    
    # Plot audio preview
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 6))
    
    # Time domain plot (first few channels)
    t = np.linspace(0, duration, len(audio_data))
    for i in range(min(4, audio_data.shape[1])):
        ax1.plot(t[:1000], audio_data[:1000, i], alpha=0.7, label=f'Mic {i+1}')
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel('Amplitude')
    ax1.set_title('Audio Waveforms (First 1000 samples)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Cross-correlation between channels
    correlations = []
    for i in range(1, min(8, audio_data.shape[1])):
        corr = np.correlate(audio_data[:5000, 0], audio_data[:5000, i], mode='full')
        correlations.append(corr)
    
    if correlations:
        corr_array = np.array(correlations)
        im = ax2.imshow(corr_array, aspect='auto', cmap='viridis')
        ax2.set_xlabel('Lag (samples)')
        ax2.set_ylabel('Channel Pair')
        ax2.set_title('Cross-correlation between Channels')
        plt.colorbar(im, ax=ax2)
    
    plt.tight_layout()
    plt.show()

## 3. Beam Focusing Computation

In [ ]:
# Interactive beam focusing parameters
@interact(
    time_window=widgets.FloatSlider(value=0.1, min=0.05, max=0.5, step=0.05, description='Time Window (s):'),
    overlap=widgets.FloatSlider(value=0.5, min=0.0, max=0.9, step=0.1, description='Overlap:'),
    compute_focus=widgets.Checkbox(value=False, description='Compute Focus')
)
def compute_beam_focusing(time_window, overlap, compute_focus):
    global energy_maps, time_stamps
    
    if not compute_focus:
        print('Check "Compute Focus" to run the computation')
        return
    
    if 'audio_data' not in globals():
        print('Please generate audio data first!')
        return
    
    print(f'Computing beam focus with {time_window}s windows, {overlap*100}% overlap...')
    
    import time
    start_time = time.time()
    
    # Compute beam focusing
    energy_maps, time_stamps = simulator.compute_beam_focus(
        audio_data, 
        time_window=time_window,
        overlap=overlap
    )
    
    computation_time = time.time() - start_time
    
    # Show results
    print(f'Computation completed in {computation_time:.2f} seconds')
    print(f'Generated {len(energy_maps)} energy maps')
    print(f'Time range: {time_stamps[0]:.2f}s to {time_stamps[-1]:.2f}s')
    print(f'Max energy: {np.max(energy_maps):.4f}')
    print(f'Real-time factor: {(time_stamps[-1] - time_stamps[0]) / computation_time:.2f}x')
    
    # Quick visualization
    fig, axes = plt.subplots(2, 3, figsize=(15, 8))
    
    # Show first, middle, and last energy maps
    indices = [0, len(energy_maps)//2, -1]
    titles = ['First', 'Middle', 'Last']
    
    extent = [-simulator.target_size/2, simulator.target_size/2, 
             -simulator.target_size/2, simulator.target_size/2]
    
    for i, (idx, title) in enumerate(zip(indices, titles)):
        im = axes[0, i].imshow(energy_maps[idx], extent=extent, origin='lower', 
                              cmap='hot', interpolation='bilinear')
        axes[0, i].set_title(f'{title} Energy Map (t={time_stamps[idx]:.2f}s)')
        axes[0, i].set_xlabel('X (m)')
        axes[0, i].set_ylabel('Y (m)')
        
        # Mark source position if available
        if 'source_position' in globals():
            axes[0, i].scatter(source_position[0], source_position[1], 
                             c='white', s=100, marker='x', linewidths=3)
        
        plt.colorbar(im, ax=axes[0, i])
    
    # Analysis plots
    # Max energy over time
    max_energy_over_time = np.max(energy_maps.reshape(len(energy_maps), -1), axis=1)
    axes[1, 0].plot(time_stamps, max_energy_over_time, 'b-', linewidth=2)
    axes[1, 0].set_xlabel('Time (s)')
    axes[1, 0].set_ylabel('Max Energy')
    axes[1, 0].set_title('Maximum Energy vs Time')
    axes[1, 0].grid(True, alpha=0.3)
    
    # Total energy over time
    total_energy_over_time = np.sum(energy_maps.reshape(len(energy_maps), -1), axis=1)
    axes[1, 1].plot(time_stamps, total_energy_over_time, 'r-', linewidth=2)
    axes[1, 1].set_xlabel('Time (s)')
    axes[1, 1].set_ylabel('Total Energy')
    axes[1, 1].set_title('Total Energy vs Time')
    axes[1, 1].grid(True, alpha=0.3)
    
    # Energy distribution histogram
    all_energies = energy_maps.flatten()
    axes[1, 2].hist(all_energies, bins=50, alpha=0.7, color='green')
    axes[1, 2].set_xlabel('Energy')
    axes[1, 2].set_ylabel('Frequency')
    axes[1, 2].set_title('Energy Distribution')
    axes[1, 2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 4. Interactive Visualization

In [ ]:
# Interactive energy map exploration
def explore_energy_maps():
    if 'energy_maps' not in globals():
        print('Please compute beam focusing first!')
        return
    
    @interact(
        time_index=widgets.IntSlider(value=0, min=0, max=len(energy_maps)-1, 
                                    description='Time Index:'),
        colormap=widgets.Dropdown(options=['hot', 'viridis', 'plasma', 'inferno', 'jet'],
                                 value='hot', description='Colormap:'),
        show_source=widgets.Checkbox(value=True, description='Show Source'),
        show_mics=widgets.Checkbox(value=True, description='Show Microphones')
    )
    def plot_energy_map(time_index, colormap, show_source, show_mics):
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
        
        # Energy map
        extent = [-simulator.target_size/2, simulator.target_size/2, 
                 -simulator.target_size/2, simulator.target_size/2]
        
        im = ax1.imshow(energy_maps[time_index], extent=extent, origin='lower', 
                       cmap=colormap, interpolation='bilinear')
        
        ax1.set_title(f'Energy Map at t = {time_stamps[time_index]:.3f}s')
        ax1.set_xlabel('X (m)')
        ax1.set_ylabel('Y (m)')
        
        # Show source position
        if show_source and 'source_position' in globals():
            ax1.scatter(source_position[0], source_position[1], 
                       c='white', s=200, marker='x', linewidths=4, label='Source')
        
        # Show microphone positions projected to target plane
        if show_mics:
            mic_x = simulator.mic_positions[:, 0]
            mic_y = simulator.mic_positions[:, 1]
            ax1.scatter(mic_x, mic_y, c='cyan', s=30, marker='o', 
                       alpha=0.7, label='Microphones')
        
        if show_source or show_mics:
            ax1.legend()
        
        plt.colorbar(im, ax=ax1, label='Energy')
        
        # Cross-section plots
        center_idx = simulator.grid_resolution // 2
        
        # Horizontal cross-section
        x_coords = np.linspace(-simulator.target_size/2, simulator.target_size/2, 
                              simulator.grid_resolution)
        horizontal_slice = energy_maps[time_index][center_idx, :]
        ax2.plot(x_coords, horizontal_slice, 'b-', linewidth=2, label='Horizontal (Y=0)')
        
        # Vertical cross-section
        y_coords = np.linspace(-simulator.target_size/2, simulator.target_size/2, 
                              simulator.grid_resolution)
        vertical_slice = energy_maps[time_index][:, center_idx]
        ax2.plot(y_coords, vertical_slice, 'r-', linewidth=2, label='Vertical (X=0)')
        
        ax2.set_xlabel('Position (m)')
        ax2.set_ylabel('Energy')
        ax2.set_title('Cross-sections through Center')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

explore_energy_maps()

## 5. Animation and Export

In [ ]:
# Create and display animation
@interact(
    create_animation=widgets.Checkbox(value=False, description='Create Animation'),
    save_to_file=widgets.Checkbox(value=False, description='Save to File'),
    animation_speed=widgets.IntSlider(value=100, min=50, max=500, step=50, 
                                    description='Speed (ms/frame):')
)
def create_animation_widget(create_animation, save_to_file, animation_speed):
    if not create_animation:
        print('Check "Create Animation" to generate the animation')
        return
    
    if 'energy_maps' not in globals():
        print('Please compute beam focusing first!')
        return
    
    print('Creating animation...')
    
    # Create the animation using the simulator's method
    anim = simulator.visualize_energy_maps(
        energy_maps, 
        time_stamps,
        save_animation=save_to_file,
        filename='beam_focus_animation.gif'
    )
    
    if save_to_file:
        print('Animation saved as beam_focus_animation.gif')
    
    # Display in notebook
    try:
        html_anim = HTML(anim.to_jshtml())
        display(html_anim)
    except:
        print('Could not display animation inline. Check the saved file.')


## 6. Performance Analysis

In [ ]:
# Performance benchmarking
@interact(
    run_benchmark=widgets.Checkbox(value=False, description='Run Benchmark'),
    test_duration=widgets.FloatSlider(value=2.0, min=0.5, max=5.0, step=0.5, 
                                    description='Test Duration (s):')
)
def run_performance_benchmark(run_benchmark, test_duration):
    if not run_benchmark:
        print('Check "Run Benchmark" to start performance testing')
        return
    
    if 'simulator' not in globals():
        print('Please create a simulator first!')
        return
    
    print(f'Running benchmark with {test_duration}s audio...')
    
    # Run the benchmark
    results = simulator.benchmark_performance(duration=test_duration)
    
    # Create performance visualization
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    
    # Timing breakdown
    times = [results['audio_generation_time'], results['beam_focus_time']]
    labels = ['Audio Generation', 'Beam Focusing']
    colors = ['lightblue', 'lightcoral']
    
    axes[0, 0].pie(times, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
    axes[0, 0].set_title('Time Distribution')
    
    # Performance metrics
    metrics = ['Samples/sec', 'Grid Points/sec', 'Real-time Factor']
    values = [results['samples_per_second'], 
             results['grid_points_per_second'], 
             results['real_time_factor']]
    
    bars = axes[0, 1].bar(metrics, values, color=['green', 'orange', 'purple'])
    axes[0, 1].set_title('Performance Metrics')
    axes[0, 1].set_ylabel('Rate')
    
    # Add value labels on bars
    for bar, value in zip(bars, values):
        height = bar.get_height()
        axes[0, 1].text(bar.get_x() + bar.get_width()/2., height,
                        f'{value:.1f}', ha='center', va='bottom')
    
    # System configuration
    config_text = f"""Configuration:
Microphones: {simulator.n_mics}
Grid Resolution: {simulator.grid_resolution}×{simulator.grid_resolution}
Sample Rate: {simulator.sample_rate} Hz
Array Radius: {simulator.array_radius} m
Target Distance: {simulator.target_distance} m

Results:
Total Time: {results['total_time']:.3f} s
Beam Focus Time: {results['beam_focus_time']:.3f} s
Real-time Factor: {results['real_time_factor']:.2f}x
"""
    
    axes[1, 0].text(0.05, 0.95, config_text, transform=axes[1, 0].transAxes,
                    verticalalignment='top', fontfamily='monospace', fontsize=10)
    axes[1, 0].set_xlim(0, 1)
    axes[1, 0].set_ylim(0, 1)
    axes[1, 0].axis('off')
    axes[1, 0].set_title('Configuration & Results')
    
    # Performance rating
    rt_factor = results['real_time_factor']
    if rt_factor >= 10:
        rating = 'Excellent'
        color = 'green'
    elif rt_factor >= 5:
        rating = 'Very Good'
        color = 'lightgreen'
    elif rt_factor >= 1:
        rating = 'Good'
        color = 'yellow'
    elif rt_factor >= 0.5:
        rating = 'Fair'
        color = 'orange'
    else:
        rating = 'Poor'
        color = 'red'
    
    axes[1, 1].text(0.5, 0.5, f'Performance\nRating:\n{rating}\n({rt_factor:.1f}x RT)',
                    ha='center', va='center', fontsize=16, fontweight='bold',
                    bbox=dict(boxstyle='round,pad=0.5', facecolor=color, alpha=0.7))
    axes[1, 1].set_xlim(0, 1)
    axes[1, 1].set_ylim(0, 1)
    axes[1, 1].axis('off')
    axes[1, 1].set_title('Performance Rating')
    
    plt.tight_layout()
    plt.show()


## 7. Advanced Experiments

In [ ]:
# Multiple source experiment
def multiple_source_experiment():
    print('Setting up multiple source experiment...')
    
    if 'simulator' not in globals():
        print('Please create a simulator first!')
        return
    
    # Define multiple sources
    sources = [
        (-0.08, -0.08, simulator.target_distance),  # Bottom-left
        (0.08, 0.08, simulator.target_distance),    # Top-right
        (0.0, 0.0, simulator.target_distance)       # Center
    ]
    
    print(f'Generating audio with {len(sources)} sources...')
    for i, source in enumerate(sources):
        print(f'  Source {i+1}: ({source[0]:.2f}, {source[1]:.2f}, {source[2]:.2f})')
    
    # Generate multi-source audio
    multi_audio = simulator.generate_test_audio(duration=4.0, source_positions=sources)
    
    # Compute beam focusing
    print('Computing beam focus for multiple sources...')
    multi_energy_maps, multi_time_stamps = simulator.compute_beam_focus(
        multi_audio, time_window=0.15, overlap=0.7
    )
    
    # Analyze results
    print(f'Analysis complete. Generated {len(multi_energy_maps)} energy maps.')
    
    # Visualization
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    
    extent = [-simulator.target_size/2, simulator.target_size/2, 
             -simulator.target_size/2, simulator.target_size/2]
    
    # Show energy maps at different times
    time_indices = [0, len(multi_energy_maps)//3, 2*len(multi_energy_maps)//3]
    
    for i, t_idx in enumerate(time_indices):
        im = axes[0, i].imshow(multi_energy_maps[t_idx], extent=extent, 
                              origin='lower', cmap='hot', interpolation='bilinear')
        axes[0, i].set_title(f'Energy Map at t={multi_time_stamps[t_idx]:.2f}s')
        axes[0, i].set_xlabel('X (m)')
        axes[0, i].set_ylabel('Y (m)')
        
        # Mark source positions
        for j, source in enumerate(sources):
            axes[0, i].scatter(source[0], source[1], c='white', s=100, 
                             marker='x', linewidths=3, label=f'Source {j+1}' if i == 0 else '')
        
        if i == 0:
            axes[0, i].legend()
        
        plt.colorbar(im, ax=axes[0, i])
    
    # Analysis plots
    max_energy = np.max(multi_energy_maps.reshape(len(multi_energy_maps), -1), axis=1)
    total_energy = np.sum(multi_energy_maps.reshape(len(multi_energy_maps), -1), axis=1)
    
    axes[1, 0].plot(multi_time_stamps, max_energy, 'b-', linewidth=2)
    axes[1, 0].set_xlabel('Time (s)')
    axes[1, 0].set_ylabel('Max Energy')
    axes[1, 0].set_title('Maximum Energy vs Time')
    axes[1, 0].grid(True, alpha=0.3)
    
    axes[1, 1].plot(multi_time_stamps, total_energy, 'r-', linewidth=2)
    axes[1, 1].set_xlabel('Time (s)')
    axes[1, 1].set_ylabel('Total Energy')
    axes[1, 1].set_title('Total Energy vs Time')
    axes[1, 1].grid(True, alpha=0.3)
    
    # Peak tracking
    peak_positions = []
    for energy_map in multi_energy_maps:
        peak_idx = np.unravel_index(np.argmax(energy_map), energy_map.shape)
        x_coord = (peak_idx[1] / simulator.grid_resolution - 0.5) * simulator.target_size
        y_coord = (peak_idx[0] / simulator.grid_resolution - 0.5) * simulator.target_size
        peak_positions.append((x_coord, y_coord))
    
    peak_x = [pos[0] for pos in peak_positions]
    peak_y = [pos[1] for pos in peak_positions]
    
    axes[1, 2].plot(peak_x, peak_y, 'go-', markersize=4, linewidth=1, alpha=0.7)
    
    # Mark actual source positions
    for j, source in enumerate(sources):
        axes[1, 2].scatter(source[0], source[1], c='red', s=150, 
                         marker='x', linewidths=4, label=f'True Source {j+1}')
    
    axes[1, 2].set_xlabel('X Position (m)')
    axes[1, 2].set_ylabel('Y Position (m)')
    axes[1, 2].set_title('Peak Energy Tracking')
    axes[1, 2].legend()
    axes[1, 2].grid(True, alpha=0.3)
    axes[1, 2].axis('equal')
    
    plt.tight_layout()
    plt.show()
    
    return multi_energy_maps, multi_time_stamps

# Button to run experiment
experiment_button = widgets.Button(description='Run Multiple Source Experiment')
experiment_output = widgets.Output()

def on_experiment_button_clicked(b):
    with experiment_output:
        experiment_output.clear_output()
        multiple_source_experiment()

experiment_button.on_click(on_experiment_button_clicked)

display(experiment_button, experiment_output)

## 8. Summary and Export

In [ ]:
# Summary and data export
def export_results():
    """Export simulation results and configuration."""
    
    if 'energy_maps' not in globals():
        print('No results to export. Please run a simulation first.')
        return
    
    import pickle
    import json
    from datetime import datetime
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # Export configuration
    config = {
        'n_mics': simulator.n_mics,
        'array_radius': simulator.array_radius,
        'target_distance': simulator.target_distance,
        'target_size': simulator.target_size,
        'grid_resolution': simulator.grid_resolution,
        'sample_rate': simulator.sample_rate,
        'sound_speed': simulator.sound_speed,
        'timestamp': timestamp
    }
    
    with open(f'simulation_config_{timestamp}.json', 'w') as f:
        json.dump(config, f, indent=2)
    
    # Export results
    results = {
        'energy_maps': energy_maps,
        'time_stamps': time_stamps,
        'config': config
    }
    
    with open(f'simulation_results_{timestamp}.pkl', 'wb') as f:
        pickle.dump(results, f)
    
    # Export energy maps as numpy arrays
    np.save(f'energy_maps_{timestamp}.npy', energy_maps)
    np.save(f'time_stamps_{timestamp}.npy', time_stamps)
    
    print(f'Results exported with timestamp: {timestamp}')
    print(f'Files created:')
    print(f'  - simulation_config_{timestamp}.json')
    print(f'  - simulation_results_{timestamp}.pkl')
    print(f'  - energy_maps_{timestamp}.npy')
    print(f'  - time_stamps_{timestamp}.npy')

# Export button
export_button = widgets.Button(description='Export Results', button_style='success')
export_output = widgets.Output()

def on_export_button_clicked(b):
    with export_output:
        export_output.clear_output()
        export_results()

export_button.on_click(on_export_button_clicked)

display(export_button, export_output)

# Final summary
print('\n' + '='*60)
print('ULTRA-FAST AUDIO BEAM FOCUSING SIMULATOR')
print('='*60)
print('Interactive notebook for exploring beam focusing capabilities.')
print('\nFeatures demonstrated:')
print('✓ Configurable circular microphone arrays')
print('✓ FFT-based fractional delay processing')
print('✓ Real-time beam focusing computation')
print('✓ Interactive parameter adjustment')
print('✓ Energy map visualization and animation')
print('✓ Performance benchmarking')
print('✓ Multiple source scenarios')
print('✓ Data export capabilities')
print('\nUse the interactive widgets above to explore different configurations!')
